Khai báo các thư viện cần thiết

In [1]:
import os
import numpy as np

Khai báo đường dẫn tệp dữ liệu

In [2]:
data_path = 'D:/Code/python/mat3533/project/data/'

train_images_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')

test_images_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')

Hàm lấy dữ liệu

In [3]:
def get_mnist_data(images_path, labels_path, num_images, shuffle=False, _is=True, image_size=28):
    """
    This shuffle param is active when .gz is downloaded at:
    - 'http://yann.lecun.com/exdb/mnist/'
    - This function return random num_images in 60000 or 10000
    """
    # read data
    import gzip # to decompress gz (zip) file
    # open file training to read training data
    f_images = gzip.open(images_path,'r')
    # skip 16 first bytes because these are not data, only header infor
    f_images.read(16)
    # general: read num_images data samples if this parameter is set;
    # if not, read all (60000 training or 10000 test)
    real_num = num_images if not shuffle else (60000 if _is else 10000)
    # read all data to buf_images (28x28xreal_num)
    buf_images = f_images.read(image_size * image_size * real_num)
    # images
    images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)
    images = images.reshape(real_num, image_size, image_size,)
    # Read labels
    f_labels = gzip.open(labels_path,'r')
    f_labels.read(8)
    labels = np.zeros((real_num)).astype(np.int64)
    # rearrange to correspond the images and labels
    for i in range(0, real_num):
        buf_labels = f_labels.read(1)
        labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)

    # shuffle to get random images data
    if shuffle is True:
        rand_id = np.random.randint(real_num, size=num_images)
        images = images[rand_id, :]
        labels = labels[rand_id,]
    # change images data to type of vector 28x28 dimentional
    images = images.reshape(num_images, image_size * image_size)
    return images, labels

Lấy dữ liệu quan sát và nhãn tương ứng từ tệp dữ liệu

In [4]:
images, labels = get_mnist_data(
    train_images_path, train_labels_path, 5000, shuffle=True)
print(images.shape, labels.shape)

(5000, 784) (5000,)


Chuẩn hóa dữ liệu

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(images)
print(X.shape)

(5000, 784)


Giảm số chiều dữ liệu xuống 100 chiều

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
X_pca = pca.fit_transform(X)
y = labels
print(X_pca.shape)
print(y.shape)

(5000, 100)
(5000,)


Tạo dữ liệu train, test

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.30, random_state=42)

Xây dựng mô hình ann

In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()

model.add(Flatten(input_shape = (100,)))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(10,activation = 'softmax')) 

model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer = 'Adam', 
              metrics = ['accuracy'])

model.fit(X_train,y_train,epochs= 10 , validation_split = .2)



Epoch 1/10


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3737 - loss: 1.8402 - val_accuracy: 0.8214 - val_loss: 0.6850
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8769 - loss: 0.4793 - val_accuracy: 0.8786 - val_loss: 0.4682
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9280 - loss: 0.2544 - val_accuracy: 0.8800 - val_loss: 0.4188
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9576 - loss: 0.1729 - val_accuracy: 0.8943 - val_loss: 0.3769
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9665 - loss: 0.1206 - val_accuracy: 0.8914 - val_loss: 0.4027
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9855 - loss: 0.0763 - val_accuracy: 0.9000 - val_loss: 0.3735
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9867 - loss: 0.0568 - val_accuracy: 0.9057 - val_loss: 0.4057
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9924 - loss: 0.0408 - val_accuracy: 0.9100 - val_loss: 0.4103
Epo

Dự đoán dữ liệu test và tính độ chính xác của mô hình

In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis = 1)
print("Prediction: ", y_pred)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

precision = precision_score(y_test, y_pred, average='macro')
print("Precision:", precision)

recall = recall_score(y_test, y_pred, average='macro')
print("Recall:", recall)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Prediction:  [9 3 9 ... 2 5 9]
Accuracy: 0.9033333333333333
Precision: 0.9032645677172318
Recall: 0.9031223835149216
